In [21]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

def load_documents():
    document_loader = PyPDFDirectoryLoader("data/")
    return document_loader.load()

In [22]:
documents = load_documents()

In [23]:
print(documents[0])

page_content='TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION' metadata={'source': 'data\\four.pdf', 'page': 0}


In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def split_documents(documents : list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 100,
        length_function = len,
        is_separator_regex = False,
    )
    
    return text_splitter.split_documents(documents)


In [25]:
chunks = split_documents(documents)
print(chunks[0])

page_content='TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION' metadata={'source': 'data\\four.pdf', 'page': 0}


In [1]:

from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\SMC-HP\anaconda3\envs\cb\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [17]:
#run this if you want to delte the already created chroma database
'''
import os
import shutil
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)
'''


In [2]:
from langchain.vectorstores.chroma import Chroma
CHROMA_PATH = "chroma"

In [27]:
#run this if chroma database is not already created
'''
db = Chroma.from_documents(
    chunks,embeddings_model,persist_directory=CHROMA_PATH
)
'''

In [3]:
#connecting to existing database
db = Chroma(persist_directory=CHROMA_PATH,embedding_function=embeddings_model)

In [4]:
from langchain.prompts import ChatPromptTemplate


In [10]:
query = "i have a headache."

In [8]:
PROMPT_TEMPLATE ="""
Answer the questio based only on the following context:
{context}

---
Answer the question based on the above context:{question}
"""

In [11]:
results  = db.similarity_search_with_relevance_scores(query,k=10)
context_text = '\n\n---\n\n'.join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context = context_text,question=query)
#print(context_text)

In [12]:
from langchain_community.llms.ollama import Ollama

llm = Ollama(model="llama2")
response_text = llm.invoke(prompt)
print(response_text)

Unfortunately, I cannot diagnose or treat any medical conditions, including migraines. However, I can provide some general information and advice on how to manage a headache.

If you are experiencing a headache, there are several things you can try to alleviate the pain:

1. Rest and relaxation: Sometimes, the best thing to do is to rest and relax. Lie down in a quiet, dark room and try to relax. This can help reduce stress and tension, which can contribute to headaches.
2. Cold or warm compresses: Applying a cold or warm compress to the forehead or neck may help relieve tension and reduce pain.
3. Over-the-counter pain medication: Nonsteroidal anti-inflammatory drugs (NSAIDs) such as ibuprofen or acetaminophen can help reduce pain and inflammation. Always follow the recommended dosage instructions and consult with a doctor before taking any medication.
4. Hydration: Dehydration can exacerbate headaches, so drinking plenty of water throughout the day can help prevent and treat headache